<a href="https://colab.research.google.com/github/Abhishek2104200/QML/blob/main/Hybrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pennylane --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 85.9 MB/s eta 0:00:00


In [ ]:
import pennylane as qml
from pennylane import numpy as np
import torch
from sklearn.model_selection import train_test_split


/usr/local/lib/python3.11/dist-packages/pennylane/capture/capture_operators.py:33: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.4.28. You have version 0.5.2 installed. Please downgrade JAX to <=0.4.28 to avoid runtime errors.
  warnings.warn(


In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/sars_cov2_kmer_encoded.csv")


<ipython-input-4-a5facda1ac44>:2: DtypeWarning: Columns (17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/sars_cov2_kmer_encoded.csv")


In [ ]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your k-mer encoded CSV
df = pd.read_csv("/content/drive/MyDrive/sars_cov2_kmer_encoded.csv")

# Extract features and target
X = df[[col for col in df.columns if col.startswith('kmer_')]].values
y = df['target'].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Normalize features (optional, but helps with training)
X_train = X_train / X_train.max()
X_test = X_test / X_train.max()


<ipython-input-5-99610b61ed24>:6: DtypeWarning: Columns (17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/sars_cov2_kmer_encoded.csv")


In [ ]:
!pip install tqdm


In [ ]:
import pennylane as qml
from pennylane import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
import pandas as pd


In [ ]:

# Select features (k-mers only)
kmer_cols = [col for col in df.columns if col.startswith("kmer_")]
X = df[kmer_cols].values
y = df['target'].values

# Scale features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Convert to torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)


In [ ]:
n_qubits = 6  # Keep low to avoid simulator issues
dev = qml.device("default.qubit", wires=n_qubits)


In [ ]:
def circuit(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]


In [ ]:
weight_shapes = {"weights": (3, n_qubits, 3)}
qnode = qml.QNode(circuit, dev, interface="torch")
qlayer = qml.qnn.TorchLayer(qnode, weight_shapes)


In [ ]:
class QuantumClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.pre_net = nn.Linear(len(kmer_cols), n_qubits)
        self.q_layer = qlayer
        self.post_net = nn.Linear(n_qubits, 5)  # 5 classes

    def forward(self, x):
        x = self.pre_net(x)
        x = self.q_layer(x)
        x = self.post_net(x)
        return F.log_softmax(x, dim=1)


In [ ]:
model = QuantumClassifier()
loss_fn = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

epochs = 10
batch_size = 64

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(epochs):
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        pred = model(X_batch)
        loss = loss_fn(pred, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss:.4f}")


Epoch 1/10 - Loss: 597.1702
Epoch 2/10 - Loss: 273.0055
Epoch 3/10 - Loss: 236.8486
Epoch 4/10 - Loss: 218.2807
Epoch 5/10 - Loss: 208.6797
Epoch 6/10 - Loss: 199.9937
Epoch 7/10 - Loss: 199.7524
Epoch 8/10 - Loss: 193.4968
Epoch 9/10 - Loss: 190.5657
Epoch 10/10 - Loss: 189.7344


In [ ]:
with torch.no_grad():
    preds = model(X_test_tensor)
    predicted_classes = torch.argmax(preds, dim=1).cpu().numpy()
    print(classification_report(y_test, predicted_classes))


              precision    recall  f1-score   support

           0       0.80      0.89      0.84      1997
           1       0.95      0.97      0.96      2000
           2       0.98      0.97      0.98      2000
           3       0.87      0.93      0.90      2000
           4       0.92      0.74      0.82      1999

    accuracy                           0.90      9996
   macro avg       0.91      0.90      0.90      9996
weighted avg       0.91      0.90      0.90      9996

